# Deep learning based diffusive size factors

This notebook illustrates the use of the deep learning based diffusive conductance algorithm decribed [here](https://doi.org/10.1016/j.cageo.2022.105086). PoreSpy's `AI_size_factor` includes the steps for predicting the diffusive size factors of the conduit images. Note that the diffusive conductance of the conduits can be then calculated by multiplying the size factor by diffusivity of the phase. The function takes in the images of segmented porous medium and returns an array of diffusive size factors for all conduits in the image. Therefore, the framework can be applied to both one conduit image as well as a segmented image of porous medium:

<img src="https://user-images.githubusercontent.com/43128873/116145783-bd458380-a6ab-11eb-8103-fb9a93f0599c.png" alt="PS_dl" width="500"/>

# Trained model and supplementary materials
To use the `AI_size_factor`, the tained trained model or weight, and trained data distribution are required. These files based on training images of polydispersed spheres are available to download. Once the required files are downloaded, we can then pass the directory of the files to `AI_size_factor` to be used for predictions.

[trained model](link): This file includes both weights and architecture of the deep learning layers. To use this file, there is no need to build a model.

[trained model weights only](link): This file includes only weights of the deep learning layers. To use this file, a Resnet50 model will be built automatically in the `AI_size_factor`.

[trained data distribution](link): This file will be used in denormalizing predicted values based on normalized transform applied on training data.

Import necessary packages and functions:

In [ ]:
import porespy as ps
import scipy as sp
import numpy as np
import h5py
from matplotlib import pyplot as plt
from sklearn.metrics import r2_score
import os
import warnings
warnings.filterwarnings("ignore")
# path to the downloaded supplementary materials, by default current directory is used here:
path = os.getcwd()+'/'
model_dir = path+'saved_model.h5'
g_train_dir = './AI_example_data/g_train_original.hdf5'
f_in = h5py.File(g_train_dir, 'r')
g_train = f_in['g_train'][()]

## Create test image

We can create a 3D image using PoreSpy's `poly_disperese_spheres` generator and segment the image using `snow_partitioning methods`.

In [ ]:
np.random.seed(17)
shape = [150, 150, 150]
dist = sp.stats.norm(loc=7, scale=5)
im = ps.generators.polydisperse_spheres(shape=shape,
                                        porosity=0.5,
                                        dist=dist,
                                        r_min=7)
results = ps.filters.snow_partitioning(im=im.astype(bool))
regions = results['regions']
fig, ax = plt.subplots(1, 1, figsize=[4, 4])
ax.imshow(regions[:,:,20], origin='lower', interpolation='none')
ax.axis(False);

## Apply AI_size_factor

The `AI_size_factor` function has an optional arguemnt to choose the method to predict on the test images. If `tensor_wise=True`, the size factors for all conduits will be calculated once using a tensor of conduit images. By default `tensor_wise=False`, which indicates that the size factors for each conduit will be calculated separately in a for loop. To speedup the calculations, it is recommended to choose `tensor_wise=True`, as tensorflow deep learning model is designed to work with tensors. As the size of the image icreases, the runtime difference between these two options becomes larger.

In [ ]:
size_factors = ps.networks.AI_size_factor(regions, model_dir=model_dir, g_train=g_train, tensor_wise=True )

## Compare with finite difference

Assuming a diffusivity of 1, the diffusive conductance of the conduits will be equal to their size factors. Now let's compare the AI-based diffusive conductances with the conductance values calculated by finite difference method. The finite difference method results are found using the steps explained [here](https://doi.org/10.1016/j.cageo.2022.105086).

In [ ]:
f_in_pred = h5py.File('./AI_example_data/g_finite_difference.hdf5', 'r')
g_pred = f_in_pred['g_finite_difference'][()]
g_truth=size_factors
plt.figure(figsize=[4,4])
plt.xlim([0,65])
plt.ylim([0,65])
plt.plot(g_truth,g_pred,'*',[0,65],[0,65],'r')
plt.xlabel('g reference')
plt.ylabel('g prediction')
r2=r2_score(g_truth, g_pred)
print('The r2 prediction accuracy is', np.round(r2,2))

Note on runtime: A larger part of `AI_size_factors` runtime is related to extracting the pairs of conduits, which is the common step required for both AI and finite difference method. Once the data is prepared, AI Prediction on the tensor takes a smaller amount of time in contrast to finite difference method, as it was shown [here]((https://doi.org/10.1016/j.cageo.2022.105086)). 